# Flux Kafka - Phase 1+

Ce notebook visualise le flux de données dans Kafka entre les agents.

## Objectif

Comprendre comment les événements circulent dans Kafka entre les 3 agents.

In [ ]:
# Configuration
import os
import sys
import json
from pathlib import Path

# Ajouter le chemin de phase1
sys.path.insert(0, str(Path("../phase1").resolve()))

from src.shared.kafka_client import KafkaProducerClient, KafkaConsumerClient
from src.shared.models import LoanApplication

print("✅ Clients Kafka importés!")

## Visualisation du Flux

```
[Intake Agent] --produit--> [finance.loan.application.v1]
                                    |
                                    | consomme
                                    v
[Risk Agent] --produit--> [risk.scoring.result.v1]
                                |
                                | consomme
                                v
[Decision Agent] --produit--> [finance.loan.decision.v1]
```

## Étape 1 : Publier une Demande

Simulons la publication d'une demande par l'Agent Intake.

In [ ]:
# Note: Assurez-vous que Kafka est démarré (docker-compose up -d)
# et que les topics existent (python scripts/init_kafka.py)

producer = KafkaProducerClient(topic="finance.loan.application.v1")

# Créer une demande de test
application = LoanApplication(
    application_id="demo-001",
    applicant_id="CUST-001",
    amount_requested=50000,
    currency="USD",
    declared_monthly_income=5000,
    employment_status="FULL_TIME",
    existing_debts=10000,
)

# Publier
success = producer.produce(
    key=application.application_id,
    value=application.model_dump()
)

if success:
    print(f"✅ Demande publiée: {application.application_id}")
    print(f"   Topic: finance.loan.application.v1")
else:
    print("❌ Échec de publication")

producer.close()

## Étape 2 : Consommer un Message

Simulons la consommation d'un message par l'Agent Risk.

In [ ]:
# Créer un consumer
consumer = KafkaConsumerClient(
    topic="finance.loan.application.v1",
    group_id="notebook-demo",
    auto_offset_reset="earliest"
)

# Consommer un message (timeout de 5 secondes)
print("⏳ En attente d'un message...")
import time
for message in consumer.consume():
    if message is not None:
        print(f"📨 Message reçu:")
        print(f"   Key: {message.key()}")
        print(f"   Topic: {message.topic()}")
        print(f"   Partition: {message.partition()}")
        print(f"   Offset: {message.offset()}")
        print(f"   Value: {json.dumps(message.value(), indent=2)}")
        break
    time.sleep(0.5)

consumer.close()

## Résumé

Ce notebook a démontré :
1. ✅ Publication d'un événement dans Kafka
2. ✅ Consommation d'un événement depuis Kafka
3. ✅ Flux de données entre agents via topics

**Pour tester le flux complet :** Lancez les 3 agents dans des terminaux séparés et observez les messages circuler.